# nuScenes Map Expansion Tutorial

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle

from collections import defaultdict
from shapely.geometry import Point, Polygon

# Init NuScenes. Requires the dataset to be stored on disk.
from nuscenes.nuscenes import NuScenes
from nuscenes.map_expansion.map_api import NuScenesMap

In [ ]:
nusc = NuScenes(version='v1.0-trainval', \
                dataroot='../../../../data/', \
                verbose=False)

In [ ]:
so_map = NuScenesMap(dataroot='../../../../data/', \
                       map_name='singapore-onenorth')
bs_map = NuScenesMap(dataroot='../../../../data/', \
                       map_name='boston-seaport')
sh_map = NuScenesMap(dataroot='../../../../data/', \
                       map_name='singapore-hollandvillage')
sq_map = NuScenesMap(dataroot='../../../../data/', \
                       map_name='singapore-queenstown')

# dict mapping map name to map file
map_files = {'singapore-onenorth': so_map,
             'boston-seaport': bs_map,
             'singapore-hollandvillage': sh_map,
             'singapore-queenstown': sq_map}

# Extracting Trajectories

In [ ]:
# dict with person token as key and other features as values
pedestrian_details = dict()

# dict with scene number as key and trajectories and map name as values
scene_info = dict()

# initializing a dict for layer names and number of points in each layer
layer_list = so_map.layer_names
layer_list.append("white_area")
layer_dict = dict.fromkeys(layer_list, 0)

# defining the sensor to extract ego_pose from sample_data, 
# we need a sensor to get sample_data
sensor = "LIDAR_TOP"

In [ ]:
for n_scene in range(850): 
    # initialize the scene
    my_scene = nusc.scene[n_scene]
    # getting the map name
    cur_map = nusc.get('log', my_scene["log_token"])["location"]
    # entering the scene number and map name
    scene_info[str(n_scene)] = {"trajectories_x": [], "trajectories_y": [],\
                          "map_name": cur_map}

    # per scene person token database
    seen_person_tokens = []

    # first sample
    first_sample_token = my_scene['first_sample_token']
    sample = nusc.get('sample', first_sample_token)

    while True:
        for ann in sample['anns']:
            group_name = nusc.get('sample_annotation', ann)['category_name']
            if "human.pedestrian" in group_name and \
               nusc.get('sample_annotation', ann)['instance_token'] not in seen_person_tokens: 

                cur_person_token = nusc.get('sample_annotation', ann)['instance_token']
                cur_person_instance = nusc.get("instance", cur_person_token)
                nbr_samples = cur_person_instance['nbr_annotations']

                # initializing the dict with the new person token
                pedestrian_details[cur_person_token] = {"translation":[], 
                                                        "rotation":[],
                                                        "velocity":[],
                                                        "ego_translation":[],
                                                        "ego_rotation":[],
                                                        "ego_time": [],
                                                        "d_curb":[],
                                                        "height":[]}

                first_token = cur_person_instance['first_annotation_token']
                current_token = first_token

                for i in range(nbr_samples):
                    current_ann = nusc.get('sample_annotation', current_token)
                    
                    # getting the sample corresponding to this annotation to retrieve 
                    # ego details
                    annotation_sample = nusc.get('sample', current_ann['sample_token'])
                    
                    if current_ann["attribute_tokens"]:
                        current_attr = nusc.get('attribute', current_ann['attribute_tokens'][0])['name']
                        if current_attr.split(".")[1] != "sitting_lying_down":
                            # updating pedestrian details dict
                            pedestrian_details[cur_person_token]["group"] = group_name.split(".")[-1]
                            pedestrian_details[cur_person_token]["translation"].append(
                                        current_ann["translation"])
                            pedestrian_details[cur_person_token]["rotation"].append(
                                        current_ann["rotation"])
                            pedestrian_details[cur_person_token]["height"].append(
                                        current_ann["size"][2])
                            pedestrian_details[cur_person_token]["scene_no"] = n_scene
                            pedestrian_details[cur_person_token]["map_name"] = cur_map
                            # only takes velocity at a particular time step
                            pedestrian_details[cur_person_token]["velocity"].append(
                                list(nusc.box_velocity(current_token)))
                            
                            # updating ego details
                            lidar_data = nusc.get('sample_data', 
                                                  annotation_sample['data'][sensor])
                            ego_token = lidar_data['ego_pose_token']
                            ego_pose = nusc.get('ego_pose', ego_token)
                            pedestrian_details[cur_person_token]["ego_translation"].append(
                                ego_pose["translation"])
                            pedestrian_details[cur_person_token]["ego_rotation"].append(
                                ego_pose["rotation"])
                            pedestrian_details[cur_person_token]["ego_time"].append(
                                ego_pose["timestamp"])
                                
                            
                            # calculating d_curb 
                            cur_ped_x = current_ann["translation"][0]
                            cur_ped_y = current_ann["translation"][1]
                            layers_on_point_dict = map_files[cur_map].layers_on_point(
                                                           cur_ped_x, cur_ped_y)
                            
                            # d_curb if he is on the walkway, else give 0
                            if layers_on_point_dict["walkway"]:
                                # serching for road type polygon 50m around pedestrian
                                records_patch = map_files[cur_map].get_records_in_patch(
                                     (current_ann["translation"][0]-25,
                                     current_ann["translation"][1]-25,
                                     current_ann["translation"][0]+25,
                                     current_ann["translation"][1]+25), 
                                     ["lane", "road_block", "road_segment"])
                                
                                # save the closest distance to any road polygon
                                d_curb = np.inf
                                for l, pol in records_patch.items():
                                    for poli in pol:
                                        poli_token = map_files[cur_map].get(l,poli)["polygon_token"]
                                        cur_poly = map_files[cur_map].extract_polygon(poli_token)
                                        cur_point = Point(cur_ped_x, cur_ped_y)
                                        d_curb = min(d_curb, cur_point.distance(cur_poly))
                                
                                pedestrian_details[cur_person_token]["d_curb"].append(
                                        d_curb)
                            
                            else:
                                # pedestrian on the road has 0 distance to curb
                                pedestrian_details[cur_person_token]["d_curb"].append(0)
                                
                                
                                
#                             # ====== updating scene info dict =======
#                             scene_info[str(n_scene)]["trajectories_x"].append(
#                                         current_ann["translation"][0])
#                             scene_info[str(n_scene)]["trajectories_y"].append(
#                                         current_ann["translation"][1])
                            
#                             # ======= updating layer dict ========
#                             cur_layers = map_files[cur_map].layers_on_point(
#                                 current_ann["translation"][0], current_ann["translation"][1])
#                             # if no layer has any tokens
#                             if all('' == s or s.isspace() for s in cur_layers.values()):
#                                 layer_dict["white_area"] += 1
#                             # if any layer has at least one token
#                             else:
#                                 for b, v in cur_layers.items():
#                                     if v:
#                                         layer_dict[b] += 1
                                        
                    current_token = current_ann["next"]


                seen_person_tokens.append(cur_person_token)

        if sample['next'] != '':
            sample = nusc.get('sample', sample['next'])
        else:
            #last sample of the scene
            break

In [ ]:
for k, val in pedestrian_details.items():
    velocities_x = [v[0] for v in val["velocity"]]
    velocities_y = [v[1] for v in val["velocity"]]
    times = val["ego_time"]
    
    del_vx = np.diff(velocities_x)    
    del_vy = np.diff(velocities_y)
    del_time = 1e-6 * (np.diff(times))
    
    acc_x = [dx/dt for dx,dt in zip(del_vx, del_time)]
    acc_y = [dy/dt for dy,dt in zip(del_vy, del_time)]
    
    acc_x.append(acc_x[-1])
    acc_y.append(acc_y[-1])
        
    pedestrian_details[k]["acceleration_x"] = acc_x
    pedestrian_details[k]["acceleration_y"] = acc_y
    pedestrian_details[k]["del_time"] = del_time

In [ ]:
new_ped_details = {}

# extracting the ped trajs with more than 16 samples
for k, v in pedestrian_details.items():
    if len(v['translation']) > 15:
        if not np.any(np.isnan(np.array(v['velocity']))):
            cur_diffs = [round(1e-6*(t-s),1) for s, t in zip(v['ego_time'], v['ego_time'][1:])]
            if not any(i > 0.9 for i in cur_diffs):
                new_ped_details[k] = v

# Saving and Loading dictionaries

In [ ]:
# SAVING all the dict files in details folder
with open('details/pedestrian_details.pkl', 'wb') as handle:
    pickle.dump(pedestrian_details, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('details/scene_info.pkl', 'wb') as handle:
#     pickle.dump(scene_info, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('details/layer_dict.pkl', 'wb') as handle:
#     pickle.dump(layer_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# saving the new ped_details dict
with open('details/new_ped_details.pkl', 'wb') as handle:
    pickle.dump(new_ped_details, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# LOADING the necessary files
with open('details/scene_info.pkl', 'rb') as handle:
    scene_info = pickle.load(handle)
    
with open('details/pedestrian_details.pkl', 'rb') as handle:
    pedestrian_details = pickle.load(handle)
    
with open('details/new_ped_details.pkl', 'rb') as handle:
    new_ped_details = pickle.load(handle)

# Plotting

In [ ]:
# Plotting the regions of pedestrian positions in all the scenes
plt.figure(figsize=(24,14))
plt.rcParams.update({'font.size': 18})
plt.bar(list(layer_dict.keys())[3:], layer_values[3:], align='center', edgecolor='black', linewidth=1.2)
plt.xlabel("Regions of the map")
plt.ylabel("Number of pedestrian positions")
plt.xticks(rotation=40)
plt.title("Bar plot of number of positions of pedestrians in different layers of the map")
plt.savefig("images/pedestrians_region.png", bbox_inches='tight', pad_inches=1)
plt.show()

In [ ]:
# Plotting all the trajectories in every scene as an image over the map

#turning interactive plotting off
plt.ioff()

for n_scene in range(850):
    # plt.style.use('dark_background')
    ego_poses = map_files[scene_info[str(n_scene)]["map_name"]].render_egoposes_on_fancy_map(nusc, \
                    scene_tokens=[nusc.scene[n_scene]['token']], verbose=False)

    plt.scatter(scene_info[str(n_scene)]["trajectories_x"], 
                scene_info[str(n_scene)]["trajectories_y"], s=20, c='r', alpha=1.0, zorder=2)

    out_path = "images/scene_trajectories/" + str(n_scene) + "_trajectories.png"
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
# Plotting the group name frequencies for pedestrians
group_count = defaultdict(int)

for p in pedestrian_details.keys():
    if "group" in pedestrian_details[p]:
        group_count[pedestrian_details[p]["group"]] += 1
    
plt.figure(figsize=(24,14))
plt.rcParams.update({'font.size': 18})
plt.bar(range(len(group_count)), list(group_count.values()), align='center')
plt.xticks(range(len(group_count)), list(group_count.keys()))
plt.xlabel("Group name")
plt.ylabel("number of pedestrians")
plt.title("Group distribution of pedestrians")
plt.savefig("images/pedestrians_group.png", bbox_inches='tight', pad_inches=1)
plt.show()

In [ ]:
len_list = []
for k in new_ped_details.keys():
    len_list.append(len(new_ped_details[k]['translation']))
    
print("At least 16 points: ", sum(i >= 16 for i in len_list))

In [ ]:
time_difs = []

for k, v in new_ped_details.items():
    time_difs += [round(1e-6*(t-s),1) for s, t in zip(v['ego_time'], v['ego_time'][1:])]
    
values, counts = np.unique(time_difs, return_counts=True)

plt.figure(figsize=(16,18))
plt.bar(values,counts, width=0.1)
plt.xticks(values)
plt.show()

In [ ]:
pedestrian_key = list(new_ped_details.keys())[5]
ped_scene = new_ped_details[pedestrian_key]['scene_no']
ego_poses = map_files[scene_info[str(ped_scene)]["map_name"]].render_egoposes_on_fancy_map(nusc, \
                scene_tokens=[nusc.scene[ped_scene]['token']], verbose=False,
                render_egoposes=False, render_egoposes_range=False, 
                render_legend=False)
plt.scatter(np.array(new_ped_details[pedestrian_key]['ego_translation'])[:6,0], 
            np.array(new_ped_details[pedestrian_key]['ego_translation'])[:6,1], s=20, c='k', alpha=1.0, zorder=2)
plt.scatter(np.array(new_ped_details[pedestrian_key]["translation"])[:6,0], 
            np.array(new_ped_details[pedestrian_key]["translation"])[:6,1], s=20, c='r', alpha=1.0, zorder=2)
plt.show()

In [ ]:
for n_scene in range(162,163):
    ego_poses = map_files[scene_info[str(n_scene)]["map_name"]].render_egoposes_on_fancy_map(
                    nusc, scene_tokens=[nusc.scene[n_scene]['token']], verbose=False,
                    render_egoposes=True, render_egoposes_range=False, 
                    render_legend=False)

    plt.scatter(scene_info[str(n_scene)]["trajectories_x"], 
                scene_info[str(n_scene)]["trajectories_y"], s=20, c='r', alpha=1.0, zorder=2)

    out_path = "images/" + "new" + "_trajectories1.png"
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Distance analysis

In [ ]:
def calculate_dist(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

dist_list = []

for ped_key in new_ped_details.keys():
    for p, v in zip(new_ped_details[ped_key]["translation"], 
                    new_ped_details[ped_key]["ego_translation"]):
        dist_list.append(calculate_dist(p[:2], v[:2]))
        
plt.figure(figsize=(24,14))
plt.rcParams.update({'font.size': 18})
plt.hist(dist_list, bins=10, density=True, align="mid", edgecolor='black', linewidth=1.2)
plt.xlabel("Distance in (m)")
plt.ylabel("Number of occurences")
plt.title("Histogram of distances between pedestrian and ego-vehicle in dataset")
plt.savefig("images/pedestrians_distance.png", bbox_inches='tight', pad_inches=1)
plt.show()   

In [ ]:
velocities = velocities[~np.isnan(velocities).any(axis=1)]
velocities = velocities[~np.all(velocities == 0, axis=1)]

In [ ]:
velocities = np.absolute(velocities)

In [ ]:
vel_x, vel_y, vel_z = np.mean(velocities, axis=0)

In [ ]:
print("Average velocity in x direction ", vel_x)
print("Average velocity in y direction ", vel_y)
print("Average velocity in z direction ", vel_z)

# Acceleration analysis

In [ ]:
# plotting histogram of velocity_z diff
velz = []
velz_diffs = []

posz = []
posz_diffs = []

accx = []
accx_diffs = []

accy = []
accy_diffs = []


for l,z in new_ped_details.items():
    for i in np.array(z["velocity"])[:,2]:
        velz.append(i)
    for t in np.array(z["translation"])[:,2]:
        posz.append(t)
    for ax in np.array(z["acceleration_x"]):
        accx.append(ax)
    for ay in np.array(z["acceleration_y"]):
        accy.append(ay)
    
plt.figure(figsize=(24,16))
plt.rcParams.update({'font.size': 20})

plt.hist(velz, bins=20)
plt.title("Histogram of vel_z values")
plt.savefig("images/velz.png", bbox_inches='tight', pad_inches=1)    
plt.show()

plt.figure(figsize=(24,16))
plt.rcParams.update({'font.size': 20})

velz_diffs = np.diff(velz)
plt.hist(velz_diffs, bins=20)
plt.title("Histogram of vel_z difference")
plt.savefig("images/velz_diff.png", bbox_inches='tight', pad_inches=1)    
plt.show()

plt.figure(figsize=(24,16))
plt.rcParams.update({'font.size': 20})

plt.hist(accx, bins=20)
plt.title("Histogram of accx values")
plt.savefig("images/accx.png", bbox_inches='tight', pad_inches=1)    
plt.show()

plt.figure(figsize=(24,16))
plt.rcParams.update({'font.size': 20})

accx_diffs = np.diff(accx)
plt.hist(accx_diffs, bins=20)
plt.title("Histogram of accx difference")
plt.savefig("images/accx_diff.png", bbox_inches='tight', pad_inches=1)    
plt.show()

plt.figure(figsize=(24,16))
plt.rcParams.update({'font.size': 20})

plt.hist(accy, bins=20)
plt.title("Histogram of accy values")
plt.savefig("images/accy.png", bbox_inches='tight', pad_inches=1)    
plt.show()

plt.figure(figsize=(24,16))
plt.rcParams.update({'font.size': 20})

accy_diffs = np.diff(accy)
plt.hist(accy_diffs, bins=20)
plt.title("Histogram of accy difference")
plt.savefig("images/accy_diff.png", bbox_inches='tight', pad_inches=1)    
plt.show()

# Group height analysis

In [ ]:
# analyse the heights of different groups

adult_heights = []
child_heights = []
personal_mobility_heights = []
other_groups = set()

for k, val in new_ped_details.items():
    if val["group"] == "child":
        for h in val["height"]:
            child_heights.append(h)
    elif val["group"] == "personal_mobility":
        for h in val["height"]:
            personal_mobility_heights.append(h)
    else: 
        other_groups.add(val["group"])
        for h in val["height"]:
            adult_heights.append(h)

In [ ]:
plt.figure(figsize=(24,16))
plt.rcParams.update({'font.size': 20})
plt.hist(child_heights, bins=10)
plt.xlabel("Height in m")
plt.ylabel("Number of annotations")
plt.title("Height distribution of child group")
plt.savefig("images/group_height/child.png", bbox_inches='tight', pad_inches=1)    
plt.show()

plt.figure(figsize=(24,16))
plt.rcParams.update({'font.size': 20})
plt.hist(personal_mobility_heights, bins=10)
plt.xlabel("Height in m")
plt.ylabel("Number of annotations")
plt.title("Height distribution of personal_mobility group")
plt.savefig("images/group_height/personal_mobility.png", bbox_inches='tight', pad_inches=1)    
plt.show()

plt.figure(figsize=(24,16))
plt.rcParams.update({'font.size': 20})
plt.hist(adult_heights, bins=10)
plt.xlabel("Height in m")
plt.ylabel("Number of annotations")
plt.title("Height distribution of adult group")
plt.savefig("images/group_height/adult.png", bbox_inches='tight', pad_inches=1)    
plt.show()